# 課題

In [38]:
import cv2

## 1-フレーム画像から動画を再構成し、再構築した動画を保存する

In [39]:
video_name = "football_ori.mp4"
fourcc = cv2.VideoWriter_fourcc("M","P","4","V")
fps = 30
size = (1920, 1080)
writer = cv2.VideoWriter(video_name, fourcc, fps, size)

In [40]:
num = 1
while num<248:
    img = cv2.imread("Football\\" + str(num) + ".jpg") #1から順にread
        
    writer.write(img)
    num += 1
    
writer.release()

## 2-フレーム画像を幅720,⾼さ480の動画に再構成、保存する

In [41]:
video_name = "football_resize.mp4"
fourcc = cv2.VideoWriter_fourcc("M","P","4","V")
fps = 30
size = (720, 480)
writer = cv2.VideoWriter(video_name, fourcc, fps, size)

In [42]:
num = 1
while num<248:
    img = cv2.imread("Football\\" + str(num) + ".jpg")
    img_resize = cv2.resize(img, (720, 480)) #resizeしてwrite
        
    writer.write(img_resize)
    num += 1
    
writer.release()

## 3-fourccをH264にして保存する

In [43]:
video_name = "football_h264.mp4"
fourcc = cv2.VideoWriter_fourcc("H","2","6","4")
fps = 30
size = (1920, 1080)
writer = cv2.VideoWriter(video_name, fourcc, fps, size)

In [44]:
num = 1
while num<248:
    img = cv2.imread("Football\\" + str(num) + ".jpg")
        
    writer.write(img)
    num += 1
    
writer.release()

## 4-前回の⼿のひらを検出するプログラムをコピー・修正し、10FPS、1FPSで保存

In [45]:
import numpy as np

In [90]:
img = cv2.imread("detect\\hand3.jpg")

img_blur = cv2.blur(img, (9, 9))
img_hsv = cv2.cvtColor(img_blur, cv2.COLOR_BGR2HSV)

h, s, v = img_hsv[500, 900]

lower_range = np.array([h-3, s-30, v-50])
upper_range = np.array([h+3, s+30, v+50])
#hの範囲を変える

In [85]:
def hand_detect(img, lower_range, upper_range):
    img_blur = cv2.blur(img, (5, 5))
    img_hsv = cv2.cvtColor(img_blur, cv2.COLOR_BGR2HSV)
    
    img_range = cv2.inRange(img_hsv, lower_range, upper_range)
    
    rect_ker = np.ones((3, 3))
    #先に膨張すると腕部分を誤認識するので、先に収縮する
    img_erode = cv2.erode(img_range, rect_ker, iterations=3)
    img_dilate = cv2.dilate(img_erode, rect_ker, iterations=3)
    contours, hier = cv2.findContours(img_erode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for each_contour in contours:
        x, y, w, h = cv2.boundingRect(each_contour)
        
        if (w * h >= 5000): #面積5000以上で検出
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 3)
            
    return(img)
#前回のコピー

In [91]:
fps = int(input("FPSを入力してください"))
video_name = "hand_detect_" + str(fps) + "fps.mp4"
fourcc = cv2.VideoWriter_fourcc("M","P","4","V")
size = (640, 480)
writer = cv2.VideoWriter(video_name, fourcc, fps, size)

FPSを入力してください 10


In [92]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()
    if ret is None:
        print("capture error")
        break
        
    image = hand_detect(img, lower_range, upper_range)
    
    writer.write(image)
    cv2.imshow("image", image)
    
    waitsec = int(1000//fps)
    key = cv2.waitKey(waitsec)
    if key == 27:
        break
        
cap.release()
writer.release()
cv2.destroyAllWindows()

## 5-"a"キーが押されたら5秒巻き戻し、"d"キーが押されたら5秒スキップするプログラムの作成

In [93]:
cap = cv2.VideoCapture("hand_detect_10fps.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)

while True:
    ret, frame = cap.read()
    frame_count = cap.get(cv2.CAP_PROP_POS_FRAMES)
    print(f"\r{frame_count}", end="")
    if ret == False:
        break
    cv2.putText(frame, str(frame_count), (400, 50), cv2.FONT_HERSHEY_TRIPLEX, 2, (0, 0, 0), 5)
    
    cv2.imshow("Video", frame)
    
    wait_sec = int(1000//fps)
    key = cv2.waitKey(wait_sec)
    if key == ord('a'):
        frame_count -= (fps*5)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count) #5秒巻き戻す
    if key == ord('d'):
        frame_count += (fps*5)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count) #5秒スキップ
    #fps*5 -> 5秒分フレームを変える
        
cap.release()
cv2.destroyAllWindows()

262.0